In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install rouge
!pip install nltk
!pip install peft
!pip install bitsandbytes

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=486537cb78806f931c25724c3e531db8712caaba993899b4a2d2d4645e5d7a64
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.2 MB/s eta 0:00:0000:0100:01


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, BitsAndBytesConfig
from rouge import Rouge
from itertools import islice
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType

import numpy as np
import torch

In [3]:
dataset = load_dataset("knkarthick/dialogsum", trust_remote_code=True)

README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [4]:
print(dataset['train'])
size = len(dataset['train'])

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 12460
})


In [5]:
print(dataset['train'][0])
print()
print(dataset['train'][0]['dialogue'])
print()
print(dataset['train'][0]['summary'])
print(size)

{'id': 'train_0', 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'summary': "Mr. Smith's 

In [6]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,                                                                                       
                                              device_map="auto",
                                              quantization_config=BitsAndBytesConfig(
                                                  load_in_4bit=True,
                                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                                  bnb_4bit_quant_type="nf4",
                                              ),
                                              torch_dtype=torch.bfloat16,)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda")
model.to(device)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear4bit(in_features=768, out_features=768, bias=False)
              (k): Linear4bit(in_features=768, out_features=768, bias=False)
              (v): Linear4bit(in_features=768, out_features=768, bias=False)
              (o): Linear4bit(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear4bit(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear4bit(in_features=768, out_features=2048, bias=Fa

In [7]:
from peft import TaskType

model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)
model = get_peft_model(model, config)
model.config.use_cache = False

In [9]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Trainable model parameters: {trainable_model_params}\nAll model parameters: {all_model_params}\nPercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

Trainable model parameters: 3538944
All model parameters: 170900736
Percentage of trainable model parameters: 2.07%


In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


# **QLoRA with Zero Shot Learning**

In [ ]:
# Total number of examples and batch size
total_examples = len(dataset['train']) #in the final one replace this with the total size of the training set
batch_size = 32
dash_line = '-' * 100
zero_shot_summaries_batch = []

# Generate example indices for the full dataset
example_indices = range(total_examples)

# Function to yield batches
def batched_indices(iterable, size):
    iterator = iter(iterable)
    for first in iterator:
        yield [first] + list(islice(iterator, size - 1))

# Process each batch of indices
for batch_num, batch in enumerate(batched_indices(example_indices, batch_size)):
    print(f'{batch_num + 1}', end=" ")

    # Collect dialogues and summaries for the batch
    dialogues = [dataset['train'][index]['dialogue'] for index in batch]
    summaries = [dataset['train'][index]['summary'] for index in batch]

    # Construct prompts for each dialogue in the batch
    prompts = [f"""
      Summarize the following conversation.

      {dialogue}

      Summary:
    """ for dialogue in dialogues]

    # Tokenize all prompts in the batch simultaneously
    # inputs = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True)
    inputs = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True).to(device)

    # Generate summaries for all prompts in the batch at once
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=50)

    # Decode and print each example in the batch
    for i, (prompt, summary, output) in enumerate(zip(prompts, summaries, outputs)):
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)
        zero_shot_summaries_batch.append(decoded_output)

    # Optional: Break after a certain number of batches for testing (remove or comment for full run)
    # if batch_num >= some_value:
    #     break
for i in zero_shot_summaries_batch:
    print(i)

In [11]:
from rouge_score import rouge_scorer
import numpy as np

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Lists to store the scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Calculate ROUGE scores for each pair of predicted and reference summaries
for pred, ref in zip(zero_shot_summaries_batch, dataset['train']['summary']):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

# Calculate the average ROUGE scores across all pairs
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

# Display results
print("Average ROUGE-1 F1 Score:", avg_rouge1)
print("Average ROUGE-2 F1 Score:", avg_rouge2)
print("Average ROUGE-L F1 Score:", avg_rougeL)


Average ROUGE-1 F1 Score: 0.24673933342070553
Average ROUGE-2 F1 Score: 0.08044069285793093
Average ROUGE-L F1 Score: 0.20868851472418784


In [13]:
# Total number of examples and batch size
total_examples = len(dataset['train']) #in the final one replace this with the total size of the training set
batch_size = 16
dash_line = '-' * 100
zero_shot_changed_summaries_batch = []

# Generate example indices for the full dataset
example_indices = range(total_examples)

# Function to yield batches
def batched_indices(iterable, size):
    iterator = iter(iterable)
    for first in iterator:
        yield [first] + list(islice(iterator, size - 1))

# Process each batch of indices
for batch_num, batch in enumerate(batched_indices(example_indices, batch_size)):
    print(batch_num + 1, end=" ")

    # Collect dialogues and summaries for the batch
    dialogues = [dataset['train'][index]['dialogue'] for index in batch]
    summaries = [dataset['train'][index]['summary'] for index in batch]

    # Construct prompts for each dialogue in the batch
    prompts = [f"""
      Dialogue

      {dialogue}

      What was going on?
    """ for dialogue in dialogues]

    # Tokenize all prompts in the batch simultaneously
    # inputs = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True)
    inputs = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True).to(device)

    # Generate summaries for all prompts in the batch at once
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=100)

    # Decode and print each example in the batch
    for i, (prompt, summary, output) in enumerate(zip(prompts, summaries, outputs)):
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)
        zero_shot_changed_summaries_batch.append(decoded_output)

    # Optional: Break after a certain number of batches for testing (remove or comment for full run)
    # if batch_num >= some_value:
    #     break
for i in zero_shot_changed_summaries_batch:
    print(i)

723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 Doctor Hawkins is here to see Person1 and Person2 about their health.
Ricky is 14 months old and due for Hepatitis A, Chickenpox and Measles shots.
Person1 finds five keys and a small foot ornament.
Person1 is not interested in a girlfriend.
Malik and Wen will dance tonight.
Aims is celebrating his birthday.
The cost of the first two kilometers is two dollars and twenty cents for each additional two hundred meters.
Judy Liao has received her resume.
The computer package is good, but the monitor is not.
Person1 is applying to study abroad. He has an interview with a visa official today.
The store is full of sugar, oranges, and milk.
The photo of Mom in her cap and gown is very nice.
Bean is a sex harasser.
Keith's father retired last week. He wants to spend more 

In [15]:
from rouge_score import rouge_scorer
import numpy as np

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Lists to store the scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Calculate ROUGE scores for each pair of predicted and reference summaries
for pred, ref in zip(zero_shot_changed_summaries_batch, dataset['train']['summary']):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

# Calculate the average ROUGE scores across all pairs
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

# Display results
print("Average ROUGE-1 F1 Score:", avg_rouge1)
print("Average ROUGE-2 F1 Score:", avg_rouge2)
print("Average ROUGE-L F1 Score:", avg_rougeL)


Average ROUGE-1 F1 Score: 0.296321532019658
Average ROUGE-2 F1 Score: 0.11551223115756747
Average ROUGE-L F1 Score: 0.24870394272888446
